# Importando informações de segurança do website da SSP

Bibliotecas usadas:
- Pandas
- Selenium
- BeautifulSoup

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

Abrir o navegador do Google Chrome

In [2]:
driver = webdriver.Chrome("C:/Users/rafael.castelao/chromedriver_win32/chromedriver.exe")

Abrir a página do SSP

In [3]:
driver.get('http://www.ssp.sp.gov.br/Estatistica/Pesquisa.aspx')

Selecionar as opões desejadas para cada campo drop-down

In [4]:
# Campo 'Ano' = '2018'
driver.find_element_by_xpath("//select[@id='conteudo_ddlAnos']/option[text()='2018']").click()

# Campo 'Regiões' = 'Capital'
driver.find_element_by_xpath("//select[@id='conteudo_ddlRegioes']/option[text()='Capital']").click()

# Campo 'Municípios' = 'São Paulo'
driver.find_element_by_xpath("//select[@id='conteudo_ddlMunicipios']/option[text()='São Paulo']").click()

Selecionar a opção "Ocorrências Registradas por Mês"

In [5]:
driver.find_element_by_id('conteudo_btnMensal').click()

Obtendo todas as opções do campo drop-down "Delegacias"

In [6]:
delegaciasBox = driver.find_element_by_id("conteudo_ddlDelegacias")
delegaciasOptions = [x for x in delegaciasBox.find_elements_by_tag_name("option")]

# Armazenar os nomes das delegacias em um array de strings
# Isso evita que seja feita uma busca no array 'delegaciasOptions', enquanto a página é atualizada
#    Como esse array carrega informações de elementos html, eles precisam existir no DOM quando são consultados
delegaciaName = []
for element in delegaciasOptions:
    delegaciaName.append(element.get_attribute("text"))

Abrir o arquivo Excel para armazenamento das informações

In [7]:
writer = pd.ExcelWriter('SSP_info.xlsx', engine='xlsxwriter')

Armazenar a tabela de ocorrências para cada delegacia em abas separadas no arquivo Excel

In [8]:
for element in delegaciaName:
    # Seleciona a delegacia no campo drop-down
    optionNameXpath = "//select[@id='conteudo_ddlDelegacias']/option[text()='" + element + "']"
    driver.find_element_by_xpath(optionNameXpath).click()
    
    # A biblioteca Beautiful Soup obtém as informações através do DOM
    soup_level2 = BeautifulSoup(driver.page_source, 'lxml')
    
    # Carrega a tabela HTML da página
    table = soup_level2.find_all('table')[0]
    
    # Transforma a tabela HTML em um DataFrame
    df = pd.read_html(str(table), header=0)
    
    # Escreve o DataFrame em uma aba do arquivo Excel
    df[0].to_excel(writer, sheet_name=element[:30])

Salva o arquivo Excel, que se encontra na raiz da pasta

In [9]:
writer.save()